In [1]:
import rasterio
import numpy as np 

In [2]:
fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_dem_egm_void.tif"

In [3]:
# Read DEM and replace nodata values with NaN
def read_dem(dem_path):
    with rasterio.open(dem_path) as src:
        dem_data = src.read(1)
        nodata_value = src.nodata
    if nodata_value is not None:
        dem_data = dem_data.astype("float32")
        dem_data[dem_data == nodata_value] = np.nan
        dem_data[dem_data <= -30] = np.nan
    return dem_data

# Get mask for NaN values
def get_null_mask(dem_data):
    return np.isnan(dem_data)

In [5]:
import numpy as np
import xgboost as xgb

# Assuming read_dem and get_null_mask are already defined
data = read_dem(fn)  # Read DEM data
mask = get_null_mask(data)  # Get mask for null/missing values

# Coordinates and values where data is available
available_coords = np.array(np.nonzero(~mask)).T  # Coordinates where data is available
available_values = data[~mask]  # Values where data is available

# Coordinates where data is missing
missing_coords = np.array(np.nonzero(mask)).T  # Coordinates where data is missing

# Remove NaN values from available data
valid_mask = ~np.isnan(available_values)
available_coords = available_coords[valid_mask]
available_values = available_values[valid_mask]

# Convert data to DMatrix (XGBoost's optimized data structure)
dtrain = xgb.DMatrix(available_coords, label=available_values)

# Set up XGBoost parameters for GPU
params = {
    'tree_method': 'gpu_hist',  # Use GPU for histogram-based tree construction
    'predictor': 'gpu_predictor',  # Use GPU for prediction
    'objective': 'reg:squarederror',  # Regression task with squared error
    'eval_metric': 'rmse',  # Root Mean Squared Error for evaluation
    'gpu_id': 0  # Use the first GPU (if you have multiple GPUs)
}

# Train the XGBoost model
model = xgb.train(params, dtrain, num_boost_round=100)  # Adjust num_boost_round as needed
print("Finished Training")

# Predict missing values
dtest = xgb.DMatrix(missing_coords)
predicted_values = model.predict(dtest)

# Fill the missing values in the original data
data[mask] = predicted_values

# Now `data` contains the original values plus the predicted values for the missing parts

/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:03:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:03:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:03:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/learner.cc:740: 
Parameters: { "

Finished Training


/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:03:17] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [7]:
import numpy as np
import xgboost as xgb
import rasterio

def fill_missing_values_with_xgboost(fn, output_fn='filled_dem.tif', num_boost_round=100):
    """
    Fills missing values in a raster using XGBoost with GPU support and writes the result to a new file.

    Parameters:
        fn (str): Path to the input raster file.
        output_fn (str): Path to the output raster file (default: 'filled_dem.tif').
        num_boost_round (int): Number of boosting rounds for XGBoost (default: 100).

    Returns:
        None
    """
    # Read the original raster data and metadata
    with rasterio.open(fn) as src:
        data = src.read(1)  # Read the first band
        meta = src.meta

    # Get mask for null/missing values
    mask = np.isnan(data)  # Assuming missing values are NaN

    # Coordinates and values where data is available
    available_coords = np.array(np.nonzero(~mask)).T  # Coordinates where data is available
    available_values = data[~mask]  # Values where data is available

    # Coordinates where data is missing
    missing_coords = np.array(np.nonzero(mask)).T  # Coordinates where data is missing

    # Remove NaN values from available data (if any)
    valid_mask = ~np.isnan(available_values)
    available_coords = available_coords[valid_mask]
    available_values = available_values[valid_mask]

    # Convert data to DMatrix (XGBoost's optimized data structure)
    dtrain = xgb.DMatrix(available_coords, label=available_values)

    # Set up XGBoost parameters for GPU
    params = {
        'tree_method': 'gpu_hist',  # Use GPU for histogram-based tree construction
        'predictor': 'gpu_predictor',  # Use GPU for prediction
        'objective': 'reg:squarederror',  # Regression task with squared error
        'eval_metric': 'rmse',  # Root Mean Squared Error for evaluation
        'gpu_id': 0  # Use the first GPU (if you have multiple GPUs)
    }

    # Train the XGBoost model
    model = xgb.train(params, dtrain, num_boost_round=num_boost_round)
    print("Finished Training")

    # Predict missing values
    dtest = xgb.DMatrix(missing_coords)
    predicted_values = model.predict(dtest)

    # Fill the missing values in the original data
    data[mask] = predicted_values

    # Update the metadata to reflect the new data
    meta.update(dtype=data.dtype, count=1)

    # Write the filled data to a new file
    with rasterio.open(output_fn, 'w', **meta) as dst:
        dst.write(data, 1)  # Write the filled data to the first band
    print(f"Filled raster saved to {output_fn}")

# # Example usage
# fill_missing_values_with_xgboost('input_dem.tif', output_fn='filled_dem.tif')

In [26]:
import numpy as np
import xgboost as xgb
import rasterio
import os

def fill_missing_values_with_xgboost(fn, output_fn='filled_dem.tif', model_fn='xgboost_model.model', num_boost_round=100):
    """
    Fills missing values in a raster using XGBoost with GPU support and writes the result to a new file.
    Saves the trained model to disk for reuse. Skips processing if the output file already exists.

    Parameters:
        fn (str): Path to the input raster file.
        output_fn (str): Path to the output raster file (default: 'filled_dem.tif').
        model_fn (str): Path to save/load the XGBoost model (default: 'xgboost_model.model').
        num_boost_round (int): Number of boosting rounds for XGBoost (default: 100).

    Returns:
        None
    """
    #model_fn = model_fn.replace('.model', f'_{num_boost_round}.model')
    print(model_fn)
    # Check if the output file already exists
    if os.path.exists(output_fn):
        print(f"Output file {output_fn} already exists. Skipping processing.")
        return

    # Check if the model already exists
    if os.path.exists(model_fn):
        print(f"Model file {model_fn} found. Loading model...")
        model = xgb.Booster()
        model.load_model(model_fn)
        print("Model loaded.")
    else:
        # Read the original raster data and metadata
        with rasterio.open(fn) as src:
            data = src.read(1)  # Read the first band
            meta = src.meta

        # Get mask for null/missing values
        mask = np.isnan(data)  # Assuming missing values are NaN

        # Coordinates and values where data is available
        available_coords = np.array(np.nonzero(~mask)).T  # Coordinates where data is available
        available_values = data[~mask]  # Values where data is available

        # Coordinates where data is missing
        missing_coords = np.array(np.nonzero(mask)).T  # Coordinates where data is missing

        # Remove NaN values from available data (if any)
        valid_mask = ~np.isnan(available_values)
        available_coords = available_coords[valid_mask]
        available_values = available_values[valid_mask]

        # Convert data to DMatrix (XGBoost's optimized data structure)
        dtrain = xgb.DMatrix(available_coords, label=available_values)

        # Set up XGBoost parameters for GPU
        params = {
            'tree_method': 'gpu_hist',  # Use GPU for histogram-based tree construction
            'predictor': 'gpu_predictor',  # Use GPU for prediction
            'objective': 'reg:squarederror',  # Regression task with squared error
            'eval_metric': 'rmse',  # Root Mean Squared Error for evaluation
            'gpu_id': 0  # Use the first GPU (if you have multiple GPUs)
        }

        # Train the XGBoost model
        print("Training model...")
        model = xgb.train(params, dtrain, num_boost_round=num_boost_round)
        print("Finished Training")

        

        # Save the trained model to disk
        
        model.save_model(model_fn)
        print(f"Model saved to {model_fn}")

    # If the model was loaded, read the original raster data and metadata
    if not os.path.exists(output_fn):
        with rasterio.open(fn) as src:
            data = src.read(1)  # Read the first band
            meta = src.meta

        # Get mask for null/missing values
        mask = np.isnan(data)

        # Coordinates where data is missing
        missing_coords = np.array(np.nonzero(mask)).T

        # Predict missing values
        dtest = xgb.DMatrix(missing_coords)
        predicted_values = model.predict(dtest)

        # Fill the missing values in the original data
        data[mask] = predicted_values

        # Update the metadata to reflect the new data
        meta.update(dtype=data.dtype, count=1)

        # Write the filled data to a new file
        with rasterio.open(output_fn, 'w', **meta) as dst:
            dst.write(data, 1)  # Write the filled data to the first band
        print(f"Filled raster saved to {output_fn}")

# Example usage
# fill_missing_values_with_xgboost('input_dem.tif', output_fn='filled_dem.tif', model_fn='xgboost_model.model')

In [28]:
from glob import glob

In [32]:
fs = glob("/media/ljp238/12TBWolf/BRCHIEVE/TILES12/*/*tdem_dem_egm_void.tif")
num_boost_round = 5000
for fi in fs:
    fo = fi.replace('void', f'vfill{num_boost_round}')
    model = fo.replace('.tif', '_xgb.model')
    fill_missing_values_with_xgboost(fi, fo,model,num_boost_round)

/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem_egm_vfill5000_xgb.model
Training model...


/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:39:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:39:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Finished Training


/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:43:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:43:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


Model saved to /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem_egm_vfill5000_xgb.model
Filled raster saved to /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem_egm_vfill5000.tif
/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E106/N09E106_tdem_dem_egm_vfill5000_xgb.model
Training model...


...

Model saved to /media/ljp238/12TBWolf/BRCHIEVE/TILES12/S02W063/S02W063_tdem_dem_egm_vfill5000_xgb.model
Filled raster saved to /media/ljp238/12TBWolf/BRCHIEVE/TILES12/S02W063/S02W063_tdem_dem_egm_vfill5000.tif
/media/ljp238/12TBWolf/BRCHIEVE/TILES12/S02W064/S02W064_tdem_dem_egm_vfill5000_xgb.model
Training model...


/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [03:35:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [03:35:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Finished Training


/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [03:37:06] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [03:37:06] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


Model saved to /media/ljp238/12TBWolf/BRCHIEVE/TILES12/S02W064/S02W064_tdem_dem_egm_vfill5000_xgb.model
Filled raster saved to /media/ljp238/12TBWolf/BRCHIEVE/TILES12/S02W064/S02W064_tdem_dem_egm_vfill5000.tif


In [24]:
fi = fn 
num_boost_round = 5000
fo = fi.replace('void', f'vfill{num_boost_round}')
model = fo.replace('.tif', '_xgb.model')

fi,fo, model

('/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_dem_egm_void.tif',
 '/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_dem_egm_vfill5000.tif',
 '/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_dem_egm_vfill5000_xgb.model')

In [25]:
fill_missing_values_with_xgboost(fi, fo,model,num_boost_round)

/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_dem_egm_vfill5000_xgb_5000.model
Training model...


/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:30:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:30:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Finished Training


/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:35:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:35:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179637554/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


Model saved to /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_dem_egm_vfill5000_xgb_5000.model
Filled raster saved to /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_dem_egm_vfill5000.tif


In [6]:
data

array([[  2.1365528,   1.7786465,   1.8915577, ...,   4.75403  ,
          4.3618765,   5.1669865],
       [  2.4263391,   2.1307611,   1.9858809, ...,   5.137808 ,
          4.837741 ,   5.0878735],
       [  2.1990566,   1.8647404,   1.9530945, ...,   5.3207855,
          5.493015 ,   5.281972 ],
       ...,
       [362.81088  , 227.52032  , 227.52032  , ...,   2.8335152,
          2.8335152,   2.848139 ],
       [191.40996  , 227.52032  , 227.52032  , ...,   1.7725046,
          1.932004 ,   2.658976 ],
       [413.33395  , 330.65112  , 332.3492   , ...,   1.6953771,
          1.3997669,   1.8727465]], dtype=float32)

In [4]:
from sklearn.ensemble import RandomForestRegressor # the new pkg or ag 

In [ ]:
data = read_dem(fn) # use esa here 
mask = get_null_mask(data)
coords = np.array(np.nonzero(mask)).T
values = data[mask]
missing_coords = np.array(np.nonzero(~mask)).T
model = RandomForestRegressor()
values = values[~np.isnan(values)]
coords = coords[~np.isnan(values)]
model.fit(coords, values)
print("Finished Training")
predicted = model.predict(missing_coords)
data[~mask] = predicted
# modify the code so that the mode is fiitted outside of missing_coords and missing values
# use where we have the values, and the corresponginh correinates
# and the make predicions only on the missing part, and join it with the full data

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by RandomForestRegressor.

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Assuming read_dem and get_null_mask are already defined
data = read_dem(fn)  # Read DEM data
mask = get_null_mask(data)  # Get mask for null/missing values

# Coordinates and values where data is available
available_coords = np.array(np.nonzero(~mask)).T  # Coordinates where data is available
available_values = data[~mask]  # Values where data is available

# Coordinates where data is missing
missing_coords = np.array(np.nonzero(mask)).T  # Coordinates where data is missing

# Remove NaN values from available data
valid_mask = ~np.isnan(available_values)
available_coords = available_coords[valid_mask]
available_values = available_values[valid_mask]

# Train the model on available data
model = RandomForestRegressor()
model.fit(available_coords, available_values)
print("Finished Training")

# Predict missing values
predicted_values = model.predict(missing_coords)

# Fill the missing values in the original data
data[mask] = predicted_values

# Now `data` contains the original values plus the predicted values for the missing parts